## Import necessary packages and modules

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from sklearn.utils.class_weight import compute_class_weight
import numpy as np


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    shear_range=0.2,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
)

train_generator = train_datagen.flow_from_directory(
    'Training/',
    target_size=(150, 150),
    color_mode='grayscale',
    batch_size=64,
    class_mode='categorical',
    seed=7
)

test_generator = test_datagen.flow_from_directory(
    'Testing/',
    target_size=(150, 150),
    color_mode='grayscale',
    batch_size=64,
    class_mode='categorical',
    seed=7
)

# Setting Up the Model

In [ ]:
# Model architecture
model = Sequential()

# 1st layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 1)))
model.add(MaxPooling2D((2, 2)))

# 2nd layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# 3rd layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# 4th layer
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten and fully connect Layers
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

# Compile model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy', 'AUC']
)

model.summary()

In [ ]:
y_train = train_generator.classes
unique_classes = np.unique(y_train)

# Compute class weights
class_weights = compute_class_weight('balanced', classes=unique_classes, y=y_train)
class_weights_dict = dict(zip(unique_classes, class_weights))


In [ ]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1, verbose=1, min_lr=0.001)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

callbacks = [early_stopping, reduce_lr, model_checkpoint]

In [ ]:
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=test_generator,
    callbacks=callbacks,
    class_weight=class_weights_dict
)

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(3, 1, figsize=(10, 15))

points = {
    'accuracy': history.history['accuracy'],
    'val_accuracy': history.history['val_accuracy'],
    'loss': history.history['loss'],
    'val_loss': history.history['val_loss'],
    'AUC': history.history['auc'],
    'val_AUC': history.history['val_auc'],
}

# Plotting accuracy
axs[0].plot(points['accuracy'])
axs[0].plot(points['val_accuracy'])
axs[0].set_title('Model Accuracy')
axs[0].set_ylabel('Accuracy')
axs[0].set_xlabel('Epoch')
axs[0].legend(['Train', 'Test'], loc='upper left')

# Plotting loss
axs[1].plot(points['loss'])
axs[1].plot(points['val_loss'])
axs[1].set_title('Model Loss')
axs[1].set_ylabel('Loss')
axs[1].set_xlabel('Epoch')
axs[1].legend(['Train', 'Test'], loc='upper left')

# Plotting AUC
axs[2].plot(points['AUC'])
axs[2].plot(points['val_AUC'])
axs[2].set_title('Model AUC')
axs[2].set_ylabel('AUC')
axs[2].set_xlabel('Epoch')
axs[2].legend(['Train', 'Test'], loc='upper left')

plt.show()